In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub state /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
029A
980A
179A
456A
379A
/;


029A
980A
179A
456A
379A


In [3]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub mulp { my ($a, $v) = @_; return [ map { $a->[$_] * $v } 0 .. $#$a ]; }
sub stringp { my $p = $_[0] // $_; return join ',', @$p }
sub rotate90p { my $p = $_[0] // $_; return [$p->[1], -$p->[0]] }
sub rotate270p { my $p = $_[0] // $_; return [-$p->[1], $p->[0]] }
sub uniqp { my %h; @h{map { join ',', @$_ } @_} = (); map [ split ',' ], keys %h }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}
$graphql::graphql_methods{map_distance} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        # say 'wat:', Dumper $self->{arr};
        my @shape = shape($arr);

        state %cache;

        return $cache{$arr}{stringp($self->{coord})} if exists $cache{$arr};

        my @start =
            grep defined,
            map selector('where.coord')->($_),
            flatten_nd
            map_nd_indexed graphql_query(qq/{ where(value:"$params->{from_value}") { coord } }/),
            $arr;
        my %stepped;
        my @next_points = @start;
        my $dist = 0;
        while (@next_points) {
            die 'assert distance' if $dist > 25000;
            @stepped{map stringp, @next_points} = ($dist) x @next_points;
            @next_points =
                uniqp
                grep { not exists $stepped{stringp $_} }
                grep $arr->[$_->[0]][$_->[1]] ne '#',
                map adjacent_points($_, @shape),
                @next_points;
            $dist += 1;
        }
        $cache{$arr} = \%stepped;
        return $cache{$arr}{stringp $self->{coord}};
    };

CODE(0x5565592944a0)

In [4]:

sub state_iterator { my ($state, $fun, @steps) = @_; map { $state = $fun->($state) } @steps }
sub state_iterator2 { my ($state, $fun, @steps) = @_; map { $fun->(\$state) } @steps }

In [16]:

sub process_day21_part1 {
    my ($input) = @_;
    my @combos = grep $_, split "\n", $input;
    # say Dumper \@combos;
    my $keypad1 = qq/
789
456
123
 0A
/;
    my $keypad2 = qq/
 ^A
<v>
/;
    sub directional_pathfind_along_map {
        my ($map, $start, $end) = @_;
        my $full_distance = subp($start, $end);
        my @directions = state_iterator2 $start, sub {
            my ($state) = @_;
            my $p = $$state;
            my $d = subp($end, $p);
            if ($d->[0] > 0 and get_value($map, addp($p, [1,0])) ne ' ') {
                $$state = addp($p, [1,0]);
                return 'v';
            } elsif ($d->[1] < 0 and get_value($map, addp($p, [0,-1])) ne ' ') {
                $$state = addp($p, [0,-1]);
                return '<';
            } elsif ($d->[1] > 0 and get_value($map, addp($p, [0,1])) ne ' ') {
                $$state = addp($p, [0,1]);
                return '>';
            } elsif ($d->[0] < 0 and get_value($map, addp($p, [-1,0])) ne ' ') {
                $$state = addp($p, [-1,0]);
                return '^';
            } else {
                die "invalid state!";
            }
            # push @instructions, map '<', $delta->[1] .. -1;
            # push @instructions, map 'v', 1 .. $delta->[0];
            # push @instructions, map '>', 1 .. $delta->[1];
            # push @instructions, map '^', $delta->[0] .. -1;
        }, 0 .. (abs($full_distance->[0]) + abs($full_distance->[1]) - 1);
    }
    sub does_step_on_space {
        my ($map, $start, @instructions) = @_;
        my $pos = $start;
        foreach (@instructions) {
            if ($_ eq 'v') { $pos->[0] += 1; }
            elsif ($_ eq '^') { $pos->[0] -= 1; }
            elsif ($_ eq '>') { $pos->[1] += 1; }
            elsif ($_ eq '<') { $pos->[1] -= 1; }
            else { die "invalid instruction in does_step_on_space: $_" }

            return 1 if $map->[$pos->[0]][$pos->[1]] eq ' ';
        }
        return 0;
    }
    sub directional_spf_along_map {
        my ($map, $start, $end) = @_;
        my @instructions;
        my $delta = subp($end, $start);
        push @instructions, map '<', $delta->[1] .. -1;
        push @instructions, map 'v', 1 .. $delta->[0];
        push @instructions, map '>', 1 .. $delta->[1];
        push @instructions, map '^', $delta->[0] .. -1;
        return @instructions if not does_step_on_space($map, $start, @instructions);

        # my @instructions;
        # push @instructions, map 'v', 1 .. $delta->[0];
        # push @instructions, map '^', $delta->[0] .. -1;
        # push @instructions, map '<', $delta->[1] .. -1;
        # push @instructions, map '>', 1 .. $delta->[1];
        return reverse @instructions;
        
    }
    sub solve_keypad_path {
        my ($keypad, $buttons) = @_;
        timecheck;
        my $map = parse_2d_map_array($keypad);
        my %position_table =
            map { $_->{value} => stringp($_->{coord}) }
            flatten_nd
            map_nd_indexed graphql_query('{ value coord }'),
            $map;
    
        my $pos = [ split ',', $position_table{A} ];

        my @instructions;
        foreach (split '', $buttons) {
            my $target = [ split ',', $position_table{$_} ];
            # push @instructions, directional_pathfind_along_map($map, $pos, $target);
            push @instructions, directional_spf_along_map($map, $pos, $target);
            push @instructions, 'A';
            $pos = $target;
        }
        # my @instructions;
        # foreach (split '', $buttons) {
        #     my $target = [ split ',', $position_table{$_} ];
        #     my $delta = subp($target, $pos);
        #     push @instructions, map '<', $delta->[1] .. -1;
        #     push @instructions, map 'v', 1 .. $delta->[0];
        #     push @instructions, map '>', 1 .. $delta->[1];
        #     push @instructions, map '^', $delta->[0] .. -1;
        #     push @instructions, 'A';
        #         $pos = $target;
        # }
        return join '', @instructions;
    }

    sub apply_keypad_path {
        my ($keypad, $instructions) = @_;
        my %position_table =
            map { $_->{value} => stringp($_->{coord}) }
            flatten_nd
            map_nd_indexed graphql_query('{ value coord }'),
            parse_2d_map_array($keypad);
        my %inverse_position_table =
            map { stringp($_->{coord}) => $_->{value} }
            flatten_nd
            map_nd_indexed graphql_query('{ value coord }'),
            parse_2d_map_array($keypad);
    
        my $pos = [ split ',', $position_table{A} ];
        my @res;
        foreach (split '', $instructions) {
            if ($_ eq 'v') { $pos->[0] += 1; }
            elsif ($_ eq '^') { $pos->[0] -= 1; }
            elsif ($_ eq '>') { $pos->[1] += 1; }
            elsif ($_ eq '<') { $pos->[1] -= 1; }
            elsif ($_ eq 'A') { push @res, $inverse_position_table{stringp $pos} }
            else { die "invalid instruction: $_" }

            say "stepped on a space!" if $inverse_position_table{stringp $pos} eq ' ';
        }

        return join '', @res;
    }

    my @solutions = 
        map solve_keypad_path($keypad2, $_),
        map solve_keypad_path($keypad2, $_),
        map solve_keypad_path($keypad1, $_),
        @combos;

    my @other_solutions = qw/
    <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
    <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A
    <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
    <v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A
    <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
    /;
    say "debug: $combos[$_]: $solutions[$_]",
        "\n\t\t-> ", apply_keypad_path($keypad2, $solutions[$_]),
        "\n\t\t-> ", apply_keypad_path($keypad2, apply_keypad_path($keypad2, $solutions[$_])),
        "\n\t\t-> ", apply_keypad_path($keypad1, apply_keypad_path($keypad2, apply_keypad_path($keypad2, $solutions[$_]))),
        "\ndebug: other: $other_solutions[$_]",
        "\n\t\t-> ", apply_keypad_path($keypad2, $other_solutions[$_]),
        "\n\t\t-> ", apply_keypad_path($keypad2, apply_keypad_path($keypad2, $other_solutions[$_])),
        "\n\t\t-> ", apply_keypad_path($keypad1, apply_keypad_path($keypad2, apply_keypad_path($keypad2, $other_solutions[$_]))) for 0 .. $#combos;
    # say "debug: other $combos[$_]: $other_solutions[$_]",
    #     "\n\t\t-> ", apply_keypad_path($keypad2, $other_solutions[$_]),
    #     "\n\t\t-> ", apply_keypad_path($keypad2, apply_keypad_path($keypad2, $other_solutions[$_])),
    #     "\n\t\t-> ", apply_keypad_path($keypad1, apply_keypad_path($keypad2, apply_keypad_path($keypad2, $other_solutions[$_]))) for 0 .. $#combos;

        sum
        map length($solutions[$_]) * ($combos[$_] =~ s/[a-z]//gir),
        # map length($solutions[$_]),
        0 .. $#solutions;
}

timer sub {
    say Dumper process_day21_part1($practice_input);
    # v<<A>>^A<A>AvA<^AA>A<vAAA>^A
    # <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
    confirm process_day21_part1($practice_input), 126384;
}

debug: 029A: <vA<AA>>^AvAA<^A>Av<<A>>^AvA^A<vA>^Av<<A>^A>AAvA^Av<<A>A>^AAAvA<^A>A
		-> v<<A>>^A<A>AvA<^AA>A<vAAA>^A
		-> <A^A>^^AvvvA
		-> 029A
debug: other: <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
		-> v<<A>>^A<A>AvA<^AA>A<vAAA>^A
		-> <A^A>^^AvvvA
		-> 029A
debug: 980A: v<<A>>^AAAvA^A<vA<AA>>^AvAA<^A>Av<<A>A>^AAAvA<^A>A<vA>^A<A>A
		-> <AAA>Av<<A>>^A<vAAA>^AvA^A
		-> ^^^A<AvvvA>A
		-> 980A
debug: other: <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A
		-> <AAA>Av<<A>>^A<vAAA>^AvA^A
		-> ^^^A<AvvvA>A
		-> 980A
debug: 179A: v<<A>>^A<vA<A>>^AAvAA<^A>Av<<A>>^AAvA^A<vA>^AA<A>Av<<A>A>^AAAvA<^A>A
		-> <Av<AA>>^A<AA>AvAA^A<vAAA>^A
		-> ^<<A^^A>>AvvvA
		-> 179A
debug: other: <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
		-> <Av<AA>>^A<AA>AvAA^A<vAAA>^A
		-> ^<<A^^A>>AvvvA
		-> 179A
debug: 456A: v<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>Av<<A>A>^AAvA<^A>A
		-> <AAv<AA>>^AvA^AvA^A<vAA>^A
		-> ^^<<A>A>AvvA
		-> 456A
debug: othe

0

Warning: Subroutine directional_pathfind_along_map redefined at reply input line 16.

Subroutine does_step_on_space redefined at reply input line 44.

Subroutine directional_spf_along_map redefined at reply input line 58.

Subroutine solve_keypad_path redefined at reply input line 76.

Subroutine apply_keypad_path redefined at reply input line 110.

Subroutine process_day21_part1 redefined at reply input line 2.


In [17]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/21/input');
    $res = process_day21_part1($input);
    say Dumper $res;
    confirm $res, 157908;
};

debug: 341A: v<<A>>^AvA^A<vA<AA>>^AAvA<^A>AvA^Av<<A>A>^AvA<^A>A<vA>^AAv<<A>>^AvA<^A>A
		-> <A>Av<<AA>^A>A<vA>^AvAA<A>^A
		-> ^A<<^AvA>>vA
		-> 341A
debug: other: <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
		-> v<<A>>^A<A>AvA<^AA>A<vAAA>^A
		-> <A^A>^^AvvvA
		-> 029A
debug: 803A: <vA<AA>>^AvA<^A>AAAvA^Av<<A>A>^AAAvA<^A>A<vA>^Av<<A>^A>AvA^Av<<A>A>^AvA<^A>A
		-> v<<A>^AAA>A<vAAA>^AvA<^A>A<vA>^A
		-> <^^^AvvvA>^AvA
		-> 803A
debug: other: <v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A
		-> <AAA>Av<<A>>^A<vAAA>^AvA^A
		-> ^^^A<AvvvA>A
		-> 980A
debug: 149A: v<<A>>^A<vA<A>>^AAvAA<^A>Av<<A>>^AvA^A<vA>^AAv<<A>^A>AvA^Av<<A>A>^AAAvA<^A>A
		-> <Av<AA>>^A<A>AvAA<^A>A<vAAA>^A
		-> ^<<A^A>>^AvvvA
		-> 149A
debug: other: <v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
		-> <Av<AA>>^A<AA>AvAA^A<vAAA>^A
		-> ^<<A^^A>>AvvvA
		-> 179A
debug: 683A: v<<A>>^AAvA^A<vA<AA>>^AvA<^A>AvA^Av<<A>A>^AAvA^A<A>Av<<A>A>^AvA<^A>A
		-> <AA>Av<<A>^A>A<vAA>A^A<v

0

In [8]:
say post_answer('2024/day/21/answer', 1, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 21 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [50]:


sub does_step_on_space {
    my ($map, $start, @instructions) = @_;
    my $pos = $start;
    foreach (@instructions) {
        if ($_ eq 'v') { $pos->[0] += 1; }
        elsif ($_ eq '^') { $pos->[0] -= 1; }
        elsif ($_ eq '>') { $pos->[1] += 1; }
        elsif ($_ eq '<') { $pos->[1] -= 1; }
        else { die "invalid instruction in does_step_on_space: $_" }

        return 1 if $map->[$pos->[0]][$pos->[1]] eq ' ';
    }
    return 0;
}
sub directional_spf_along_map {
    my ($map, $start, $end) = @_;
    my @instructions;
    my $delta = subp($end, $start);
    push @instructions, map '<', $delta->[1] .. -1;
    push @instructions, map 'v', 1 .. $delta->[0];
    push @instructions, map '^', $delta->[0] .. -1;
    push @instructions, map '>', 1 .. $delta->[1];
    return @instructions if not does_step_on_space($map, $start, @instructions);
    return reverse @instructions;
    
}
sub solve_keypad_path {
    my ($keypad, $buttons) = @_;
    timecheck;
    my $map = parse_2d_map_array($keypad);
    my %position_table =
        map { $_->{value} => stringp($_->{coord}) }
        flatten_nd
        map_nd_indexed graphql_query('{ value coord }'),
        $map;

    my $pos = [ split ',', $position_table{A} ];

    my @instructions;
    foreach (split '', $buttons) {
        my $target = [ split ',', $position_table{$_} ];
        # push @instructions, directional_pathfind_along_map($map, $pos, $target);
        push @instructions, directional_spf_along_map($map, $pos, $target);
        push @instructions, 'A';
        $pos = $target;
    }
    return join '', @instructions;
}

sub apply_keypad_path {
    my ($keypad, $instructions) = @_;
    my %position_table =
        map { $_->{value} => stringp($_->{coord}) }
        flatten_nd
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($keypad);
    my %inverse_position_table =
        map { stringp($_->{coord}) => $_->{value} }
        flatten_nd
        map_nd_indexed graphql_query('{ value coord }'),
        parse_2d_map_array($keypad);

    my $pos = [ split ',', $position_table{A} ];
    my @res;
    foreach (split '', $instructions) {
        if ($_ eq 'v') { $pos->[0] += 1; }
        elsif ($_ eq '^') { $pos->[0] -= 1; }
        elsif ($_ eq '>') { $pos->[1] += 1; }
        elsif ($_ eq '<') { $pos->[1] -= 1; }
        elsif ($_ eq 'A') { push @res, $inverse_position_table{stringp $pos} }
        else { die "invalid instruction: $_" }

        say "stepped on a space!" if $inverse_position_table{stringp $pos} eq ' ';
    }

    return join '', @res;
}

Warning: Subroutine does_step_on_space redefined at reply input line 3.

Subroutine directional_spf_along_map redefined at reply input line 17.

Subroutine solve_keypad_path redefined at reply input line 29.

Subroutine apply_keypad_path redefined at reply input line 52.


In [53]:
timer sub {};

our $keypad1 = qq/
789
456
123
 0A
/;
our $keypad2 = qq/
 ^A
<v>
/;
our %keypad2_table =
    map { $_ => (split 'A', solve_keypad_path($keypad2, $_), 2)[1] }
    # map { $_ => solve_keypad_path($keypad2, $_) }
    map { my $c = $_; map "$c$_", grep $_ !~ /\s/, split '', $keypad2 }
    grep $_ !~ /\s/,
    split '', $keypad2;
# %keypad2_table = (
#           '<<' => 'A',
#           '<>' => '>>A',
#           '<A' => '>>^A',
#           '<^' => '>^A', # swappable
#           '<v' => '>A',
#           '><' => '<<A',
#           '>>' => 'A',
#           '>A' => '^A',
#           '>^' => '<^A', # nonswappable
#           '>v' => '<A',
#           'A<' => 'v<<A',
#           'A>' => 'vA',
#           'AA' => 'A',
#           'A^' => '<A',
#           'Av' => '<vA', # nonswappable
#           '^<' => 'v<A',
#           '^>' => 'v>A', # swappable...
#           '^A' => '>A',
#           '^^' => 'A',
#           '^v' => 'vA',
#           'v<' => '<A',
#           'v>' => '>A',
#           'vA' => '>^A', # swappable
#           'v^' => '^A',
#           'vv' => 'A'
#         );
say Dumper \%keypad2_table;
sub recursive_solve_keypad_path {
    my ($initial, $pattern, $depth) = @_;

    state %cache;
    return $pattern if $depth eq 0;
    return $cache{"$initial$pattern$depth"} if exists $cache{"$initial$pattern$depth"};
    my $s = exists $keypad2_table{"$initial$pattern"} ? $keypad2_table{"$initial$pattern"}
            : (split 'A', solve_keypad_path($keypad2, "$initial$pattern"), 2)[1];
    my $prev = 'A';

    my $res = join '',
    map {
        my $res = recursive_solve_keypad_path($prev, $_, $depth - 1);
        $prev = $_;
        $res;
    }
    split '', $s;

    # say "debug: cache: '$initial$pattern$depth'\n\t" . $cache{"$initial$pattern$depth"} . "\n\t$res" if exists $cache{"$initial$pattern$depth"};
    $cache{"$initial$pattern$depth"} = $res;
    return $res;
}

sub recursive_solve_keypad_length {
    my ($initial, $pattern, $depth) = @_;

    state %cache;
    return length $pattern if $depth eq 0;
    return $cache{"$initial$pattern$depth"} if exists $cache{"$initial$pattern$depth"};
    my $s = exists $keypad2_table{"$initial$pattern"} ? $keypad2_table{"$initial$pattern"}
            : (split 'A', solve_keypad_path($keypad2, "$initial$pattern"), 2)[1];
    my $prev = 'A';

    my $res = sum
        map {
            my $res = recursive_solve_keypad_length($prev, $_, $depth - 1);
            $prev = $_;
            $res;
        }
        split '', $s;

    # say "debug: cache: '$initial$pattern$depth'\n\t" . $cache{"$initial$pattern$depth"} . "\n\t$res" if exists $cache{"$initial$pattern$depth"};
    $cache{"$initial$pattern$depth"} = $res;
    return $res;
}

sub process_day21_part2 {
    my ($input, $depth) = @_;
    my @combos = grep $_, split "\n", $input;
    # say Dumper \@combos;

    # \%keypad2_table

    # my @solutions = 
    #     map recursive_solve_keypad_path('A', $_, 1),
    #     # map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;


    # my @other_solutions = 
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;

    # confirm $solutions[$_], $other_solutions[$_] foreach 0 .. $#solutions;
    

    # my @solutions = 
    #     map recursive_solve_keypad_path('A', $_, 2),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;


    # my @other_solutions = 
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;

    # confirm $solutions[$_], $other_solutions[$_] foreach 0 .. $#solutions;
    

    # my @solutions = 
    #     map recursive_solve_keypad_path('A', $_, 5),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;
        
    # my @solution_lengths = 
    #     map recursive_solve_keypad_length('A', $_, 5),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;


    # my @other_solutions = 
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;

    # confirm $solutions[$_], $other_solutions[$_] foreach 0 .. $#solutions;
    # confirm $solution_lengths[$_], length $other_solutions[$_] foreach 0 .. $#solutions;
    

    # my @solution_lengths = 
    #     map recursive_solve_keypad_length('A', $_, 7),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;
    # my @other_solutions = 
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad2, $_),
    #     map solve_keypad_path($keypad1, $_),
    #     @combos;

    # confirm $solution_lengths[$_], length $other_solutions[$_] foreach 0 .. $#solutions;



    my @solutions = 
        map recursive_solve_keypad_length('A', $_, $depth),
        map solve_keypad_path($keypad1, $_),
        @combos;

        sum
        map $solutions[$_] * ($combos[$_] =~ s/[a-z]//gir),
        # map length($solutions[$_]),
        0 .. $#solutions;
}

timer sub {
    say Dumper process_day21_part2($practice_input, 25);
    # v<<A>>^A<A>AvA<^AA>A<vAAA>^A
    # <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
    confirm process_day21_part2($practice_input, 2), 126384;
    confirm process_day21_part2($practice_input, 3), 310188;
    confirm process_day21_part2($practice_input, 4), 757754;
    confirm process_day21_part2($practice_input, 5), 1881090;
}

[time] 0 seconds
$VAR1 = {
          '<<' => 'A',
          '<>' => '>>A',
          '<A' => '>>^A',
          '<^' => '>^A',
          '<v' => '>A',
          '><' => '<<A',
          '>>' => 'A',
          '>A' => '^A',
          '>^' => '<^A',
          '>v' => '<A',
          'A<' => 'v<<A',
          'A>' => 'vA',
          'AA' => 'A',
          'A^' => '<A',
          'Av' => '<vA',
          '^<' => 'v<A',
          '^>' => 'v>A',
          '^A' => '>A',
          '^^' => 'A',
          '^v' => 'vA',
          'v<' => '<A',
          'v>' => '>A',
          'vA' => '^>A',
          'v^' => '^A',
          'vv' => 'A'
        };

$VAR1 = '154115708116294';

confirmed: 126384
confirmed: 310188
confirmed: 757754
confirmed: 1881090
[time] 0 seconds


0

Warning: Subroutine recursive_solve_keypad_path redefined at reply input line 47.

Subroutine recursive_solve_keypad_length redefined at reply input line 70.

Subroutine process_day21_part2 redefined at reply input line 93.


In [55]:
my $res;
timer sub {
    my $input = get_challenge('2024/day/21/input');
    $res = process_day21_part2($input, 2);
    say Dumper $res;
    confirm $res, 157908;
    
    $res = process_day21_part2($input, 25);
    say Dumper $res;
};

$VAR1 = 157908;

confirmed: 157908
$VAR1 = '196910339808654';

[time] 0 seconds


0

In [58]:
# 90245482350618 < ? < 225901581929724
say post_answer('2024/day/21/answer', 2, $res);

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 21 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1